# Default Import

In [4]:
from ris import pysqldb
from IPython.display import Markdown, clear_output
from sqlalchemy import create_engine
import ris
import getpass
import datetime 
import pandas as pd
import numpy as np
import os
import requests

timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

Markdown('<strong>Notebook run on: {} | by {} | Using ris library version: {} | File Location: {}'.format(
    timestamp, getpass.getuser(), ris.__version__, os.getcwd()
))


<strong>Notebook run on: 2021-04-28 17:43 | by soge | Using ris library version: 2.2.1 | File Location: E:\RIS\Staff Folders\Samuel\Requests\SH\Camera_Nols

# Db connections

In [6]:
try: 
    db.params['user']
except:
    db = pysqldb.DbConnect(type='PG', server='dotdevrhpgsql01', database='ris')


Additional database connection details required:
User name (ris):soge
Password (ris)········
Database connection (PG) to ris on dotdevrhpgsql01 - user: soge 
Connection established 2020-03-11 16:17:47


In [7]:
#sip = pysqldb.DbConnect(server='dotpgsql01', database='sip', type='PG', user = db.params['user'], password = db.params['password'])
#forms = pysqldb.DbConnect(server='dot55sql01', database='forms', type='MS', user = 'arcgis', password = 'arcgis')

In [10]:
data = pd.read_excel(r'Data\Fixed_Speed_Through_Dec2020_w_Avg.xlsx', sheet_name = 'FIXED SPEED BY MONTH')

In [14]:
tmp_df = data['site_id'][1:]

In [20]:
tmp_df.columns = ['site_id']

In [22]:
tmp_df.columns

['site_id']

In [24]:
tmp_df.to_frame()

,site_id
1,KFS01P
2,KFS01P2
3,KFS02
4,KFS02P
5,KFS03
6,KFS03P
7,KFS03P2
8,KFS04
9,KFS04P
10,KFS04P2


In [25]:
data.columns

Index([u'site_id', u'FIXED SPEED NOLs - BY MONTH - NYNYB', u'Unnamed: 2',
       u'Unnamed: 3', u'Unnamed: 4', u'Unnamed: 5', u'Dec-20', u'Dec-20.1',
       u'Dec-20.2', u'Nov-20', u'Nov-20.1', u'Nov-20.2', u'Oct-20',
       u'Oct-20.1', u'Oct-20.2', u'Sep-20', u'Sep-20.1', u'Sep-20.2',
       u'Aug-20', u'Aug-20.1', u'Aug-20.2', u'Jul-20', u'Jul-20.1',
       u'Jul-20.2', u'Jun-20', u'Jun-20.1', u'Jun-20.2', u'May-20',
       u'May-20.1', u'May-20.2', u'Apr-20', u'Apr-20.1', u'Apr-20.2',
       u'Mar-20', u'Mar-20.1', u'Mar-20.2', u'Feb-20', u'Feb-20.1',
       u'Feb-20.2', u'Jan-20', u'Jan-20.1', u'Jan-20.2', u'Total', u'Total.1',
       u'Total.2'],
      dtype='object')

In [48]:
data[['site_id', 'Dec-20.2']].iloc[0]

site_id         site_id
Dec-20.2    Total\nNOLs
Name: 0, dtype: object

In [57]:
def fix_header(raw_data, col):
    if col =='site_id': # this is the header
        # get all non-header rows
        tmp_df = raw_data[col][1:]
        # rename the header
        tmp_df.columns = [col]
        # convert to df
        if type(tmp_df) == pd.Series:
            return tmp_df.to_frame()
    # get the header from the 1st row (include site id to join back later)
    new_name = raw_data[['site_id', col]].iloc[0] # assumes multiple sub columns
    # get the non header data
    tmp_df = raw_data[['site_id', col]][1:]
    # convert to df
    if type(tmp_df) == pd.Series:
        tmp_df = tmp_df.to_frame()
    # convert empty columns to empty str to join with sib header
    if not col:
        col = ''
    # create a list of the sub col headers
    new_name = list(new_name)
    # add site id and join with initial col header
    tmp_df.columns = ['site_id']+[col + ' ' + i for i in new_name if i!='site_id']
    # clean up formatting
    #print tmp_df
    tmp_df.rename(columns=lambda x:  x.replace('\n',' ').strip(), inplace=True)
    return tmp_df


def fix_df_headers(raw_data):
    # set up base df (just site ids)
    df = fix_header(raw_data, 'site_id')
    # drop the tail rows that are totals - messes up the join
    df.drop(df[df['site_id'].isnull()].index, inplace=True)
    # fix header for all other columns and join back to base df
    for c in set(raw_data.columns): # force unique
        # print c
        if c == 'site_id':
            pass
        else:
            _ = fix_header(raw_data, c)
            df = df.join(_.set_index('site_id'), on='site_id', how='left')
    return df

In [63]:
fix_df_headers(data)

,site_id,Jun-20 Avg Per Day,Feb-20.1 Days Active,Feb-20.2 Total NOLs,Aug-20 Avg Per Day,Jun-20.1 Days Active,Jun-20.2 Total NOLs,Jul-20.1 Days Active,Jul-20 Avg Per Day,Apr-20 Avg Per Day,...,Nov-20.1 Days Active,Aug-20.1 Days Active,Sep-20.1 Days Active,Sep-20.2 Total NOLs,Mar-20.1 Days Active,Apr-20.1 Days Active,Apr-20.2 Total NOLs,FIXED SPEED NOLs - BY MONTH - NYNYB LOC_DESCRIPTION,Total.2 Total NOLs,Total Avg Per Day
1,KFS01P,9,17,117,3,22,192,22,3,14,...,7,21,9,2,17,13,176,4TH AVE @ 42ND ST,1069,6
2,KFS01P2,0,0,0,119,0,0,0,0,0,...,20,1,21,2487,0,0,0,3RD AVE @ 41ST ST,7217,84
3,KFS02,87,20,1373,72,22,1915,23,85,109,...,20,21,21,925,22,23,2517,LINDEN BLVD @ ROCKAWAY AVE,17859,69
4,KFS02P,92,20,966,69,22,2024,23,80,113,...,20,21,21,1019,22,22,2477,LINDEN BLVD @ ROCKAWAY AVE,17644,69
5,KFS03,69,20,719,57,22,1510,23,76,68,...,20,21,21,1132,22,22,1501,RALPH AVE @ BEVERLEY RD,13549,53
6,KFS03P,53,19,399,48,22,1170,23,60,58,...,20,21,21,671,22,22,1281,RALPH AVE @ BEVERLEY RD,9585,38
7,KFS03P2,0,0,0,12,0,0,0,0,0,...,20,1,21,80,0,0,0,CLARENDON RD @ E 59TH ST,281,3
8,KFS04,56,20,535,53,22,1231,23,56,58,...,20,21,21,1092,22,22,1277,KINGS HWY @ E 94TH ST,11363,44
9,KFS04P,33,20,434,26,22,722,23,31,40,...,6,21,21,514,22,22,879,KINGS HWY @ E 95TH ST,6278,26
10,KFS04P2,0,0,0,112,0,0,0,0,0,...,20,1,21,2094,0,0,0,ROCKAWAY PKWY @ WILLMOHR ST,6171,72


In [55]:
fix_header(data, 'Dec-20')

       site_id Dec-20 Avg\nPer\nDay
1       KFS01P                    3
2      KFS01P2                   77
3        KFS02                   35
4       KFS02P                   36
5        KFS03                   35
6       KFS03P                   18
7      KFS03P2                    3
8        KFS04                   29
9       KFS04P                   16
10     KFS04P2                   49
11       KFS05                   66
12      KFS05P                   50
13       KFS08                   19
14      KFS08P                   20
15       KFS09                   15
16      KFS09P                    9
17       KFS10                   28
18      KFS100                    2
19      KFS101                    1
20      KFS102                   11
21     KFS102P                    2
22    KFS102P2                    2
23      KFS103                    1
24     KFS103P                   57
25    KFS103P2                   33
26      KFS104                    2
27      KFS105              

,site_id,Dec-20 Avg Per Day
1,KFS01P,3
2,KFS01P2,77
3,KFS02,35
4,KFS02P,36
5,KFS03,35
6,KFS03P,18
7,KFS03P2,3
8,KFS04,29
9,KFS04P,16
10,KFS04P2,49
